## Note
This notebook works on Kaggle Python image

In [7]:
import csv
import requests
import io
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [11]:
# Classes for data
class EarthQuake:
    def __init__(self, row):
        self.timestamp = row[0]
        self.lat = float(row[1])
        self.lon = float(row[2])
        try:
            self.magnitude = float(row[4])
        except ValueError:
            self.magnitude = 0
    def __str__(self):
        return f"Earthquake: {self.magnitude} at {self.lat},{self.lon}"

In [18]:
def get_earthquake_data(url):
    response = requests.get(url)
    csvio = io.StringIO(response.text)
    reader = csv.reader(csvio)
    header = next(reader)
    quakes = [EarthQuake(row) for row in reader]
    quakes = [q for q in quakes if q.magnitude > 0]
    return quakes

In [21]:
def get_marker(magnitude):
    markersize =  magnitude * 2.5
    if magnitude < 1.0:
        return ('bo'), markersize
    if magnitude < 3.0:
        return ('go'), markersize
    elif magnitude < 5.0:
        return ('yo'), markersize
    else:
        return ('ro'), markersize

In [30]:
def create_png(url, outfile):
    quakes = get_earthquake_data(url)
    print(quakes[0].__dict__)
    
    mpl.rcParams['figure.figsize'] = '16, 12'
    m = Basemap(projection='kav7', lon_0=-90, resolution='l', area_thresh=1000.0)
    m.drawcoastlines()
    m.drawcountries()
    m.drawmapboundary(fill_color='0.3')
    m.drawparallels(np.arange(-90.,99.,30.))
    
    start_day = quakes[-1].timestamp[:10]
    end_day = quakes[0].timestamp[:10]
    quakes.sort(key=lambda q: q.magnitude, reverse=True)
    
    for q in quakes:
        x,y = m(q.lon, q.lat)
        mcolor, msize = get_marker(q.magnitude)
        m.plot(x, y, mcolor, markersize=msize)
    
    plt.title(f"Earthquakes {start_day} to {end_day}")
    plt.savefig(outfile)

In [31]:
url = 'http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.csv'
outfile = 'earthquakes.png'
create_png(url, outfile)

{'timestamp': '2021-06-22T21:36:12.730Z', 'lat': 33.1601667, 'lon': -115.6385, 'magnitude': 1.41}
